In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [7]:
training = spark.read.csv('test3.csv', header = True, inferSchema = True)

In [5]:
training.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Tham| 22|        10| 30000|
|Long | 20|         8| 10000|
|Thien| 21|         4| 20000|
|    A| 29|         8| 25000|
|    B| 30|         3| 40000|
|    C| 40|         1| 15000|
+-----+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
# Gọi ra vecto
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['Age', 'Experience'],
                                  outputCol = 'Independent Features')

In [10]:
output = featureassembler.transform(training)
output.show()

+-----+---+----------+------+--------------------+
| Name|Age|Experience|Salary|Independent Features|
+-----+---+----------+------+--------------------+
| Tham| 22|        10| 30000|         [22.0,10.0]|
|Long | 20|         8| 10000|          [20.0,8.0]|
|Thien| 21|         4| 20000|          [21.0,4.0]|
|    A| 29|         8| 25000|          [29.0,8.0]|
|    B| 30|         3| 40000|          [30.0,3.0]|
|    C| 40|         1| 15000|          [40.0,1.0]|
+-----+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data = output.select('Independent Features', 'Salary')
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [22.0,10.0]| 30000|
|          [20.0,8.0]| 10000|
|          [21.0,4.0]| 20000|
|          [29.0,8.0]| 25000|
|          [30.0,3.0]| 40000|
|          [40.0,1.0]| 15000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
# Train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent Features',
                            labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [15]:
# Coefficients
regressor.coefficients

DenseVector([1657.0439, -374.567])

In [16]:
# Intercepts
regressor.intercept

-12959.72863741355

In [17]:
# Prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [40.0,1.0]| 15000|52947.459584295655|
+--------------------+------+------------------+

